In [1]:
#-------Import Dependencies-------#
import pandas as pd
import os,shutil,math
import numpy as np
import matplotlib.pyplot as plt
import cv2

from sklearn.utils import shuffle
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,roc_curve,auc

from PIL import Image
from PIL import ImageDraw
from glob import glob
from tqdm import tqdm
from skimage.io import imread
from IPython.display import SVG

from keras.utils.vis_utils import model_to_dot
from tensorflow.keras.applications import VGG16
from keras.applications.vgg19 import VGG19,preprocess_input
from keras.applications.xception import Xception
from keras.applications.nasnet import NASNetMobile
from keras.models import Sequential,Input,Model
from keras.layers import Dense,Flatten,Dropout,Concatenate,GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam,SGD
from keras.utils.vis_utils import plot_model
from keras.callbacks import ModelCheckpoint,EarlyStopping,TensorBoard,CSVLogger,ReduceLROnPlateau,LearningRateScheduler

Using TensorFlow backend.
c:\users\dell\anaconda3\envs\deeplearning\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\dell\anaconda3\envs\deeplearning\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\dell\anaconda3\envs\deeplearning\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\dell\anaconda3\envs\

In [2]:
input_dir='../Downloads/562468_1022626_compressed_Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/'
SAMPLE_COUNT = 1000
TESTING_BATCH_SIZE = 500
labels=pd.read_csv( input_dir +'datasets_562468_1022626_Chest_xray_Corona_Metadata.csv')

In [3]:
training_dir = input_dir + 'train/'
df = pd.DataFrame({'path': glob(os.path.join(training_dir,'*.jpeg'))})
df['id']=labels['X_ray_image_name']
df['label']=labels['Label']

In [4]:
labels

,Unnamed: 0,X_ray_image_name,Label,Dataset_type,Label_2_Virus_category,Label_1_Virus_category
0,0,IM-0128-0001.jpeg,Normal,TRAIN,NaN,NaN
1,1,IM-0127-0001.jpeg,Normal,TRAIN,NaN,NaN
2,2,IM-0125-0001.jpeg,Normal,TRAIN,NaN,NaN
3,3,IM-0122-0001.jpeg,Normal,TRAIN,NaN,NaN
4,4,IM-0119-0001.jpeg,Normal,TRAIN,NaN,NaN
...,...,...,...,...,...,...
5905,5928,person1637_virus_2834.jpeg,Pnemonia,TEST,NaN,Virus
5906,5929,person1635_virus_2831.jpeg,Pnemonia,TEST,NaN,Virus
5907,5930,person1634_virus_2830.jpeg,Pnemonia,TEST,NaN,Virus
5908,5931,person1633_virus_2829.jpeg,Pnemonia,TEST,NaN,Virus


In [5]:
df

,path,id,label
0,../Downloads/562468_1022626_compressed_Coronah...,IM-0128-0001.jpeg,Normal
1,../Downloads/562468_1022626_compressed_Coronah...,IM-0127-0001.jpeg,Normal
2,../Downloads/562468_1022626_compressed_Coronah...,IM-0125-0001.jpeg,Normal
3,../Downloads/562468_1022626_compressed_Coronah...,IM-0122-0001.jpeg,Normal
4,../Downloads/562468_1022626_compressed_Coronah...,IM-0119-0001.jpeg,Normal
...,...,...,...
5259,../Downloads/562468_1022626_compressed_Coronah...,5931B64A-7B97-485D-BE60-3F1EA76BC4F0.jpeg,Pnemonia
5260,../Downloads/562468_1022626_compressed_Coronah...,4C4DEFD8-F55D-4588-AAD6-C59017F55966.jpeg,Pnemonia
5261,../Downloads/562468_1022626_compressed_Coronah...,44C8E3D6-20DA-42E9-B33B-96FA6D6DE12F.jpeg,Pnemonia
5262,../Downloads/562468_1022626_compressed_Coronah...,446B2CB6-B572-40AB-B01F-1910CA07086A.jpeg,Pnemonia


In [4]:
negative_values = df[df.label == 'Normal'].sample(SAMPLE_COUNT)
positive_values = df[df.label == 'Pnemonia'].sample(SAMPLE_COUNT)
df = pd.concat([negative_values,positive_values]).reset_index()

In [18]:
x= df[df.label=='Normal']
y= df[df.label=='Pnemonia']

In [19]:
x.count()

path     1342
id       1342
label    1342
dtype: int64

In [20]:
y.count()

path     3922
id       3922
label    3922
dtype: int64

In [23]:
df = pd.concat([x,y]).reset_index()
df

,index,path,id,label
0,0,../Downloads/562468_1022626_compressed_Coronah...,IM-0128-0001.jpeg,Normal
1,1,../Downloads/562468_1022626_compressed_Coronah...,IM-0127-0001.jpeg,Normal
2,2,../Downloads/562468_1022626_compressed_Coronah...,IM-0125-0001.jpeg,Normal
3,3,../Downloads/562468_1022626_compressed_Coronah...,IM-0122-0001.jpeg,Normal
4,4,../Downloads/562468_1022626_compressed_Coronah...,IM-0119-0001.jpeg,Normal
...,...,...,...,...
5259,5259,../Downloads/562468_1022626_compressed_Coronah...,5931B64A-7B97-485D-BE60-3F1EA76BC4F0.jpeg,Pnemonia
5260,5260,../Downloads/562468_1022626_compressed_Coronah...,4C4DEFD8-F55D-4588-AAD6-C59017F55966.jpeg,Pnemonia
5261,5261,../Downloads/562468_1022626_compressed_Coronah...,44C8E3D6-20DA-42E9-B33B-96FA6D6DE12F.jpeg,Pnemonia
5262,5262,../Downloads/562468_1022626_compressed_Coronah...,446B2CB6-B572-40AB-B01F-1910CA07086A.jpeg,Pnemonia


In [6]:


for directory in [train_path,val_path]:
    for sub_directory in ['Normal','Pnemonia']:
        path = os.path.join(directory,sub_directory)
        os.makedirs(path,exist_ok=True)
        
train,val = train_test_split(df,train_size=0.8,stratify=df['label'])
df.set_index('id',inplace=True)

for images_paths in [(train,train_path),(val,val_path)]:
    images = images_paths[0]
    path = images_paths[1]
    for image in images['id'].values:
        file_name = image + ''
        label = str(df.loc[image,'label'])
        destination = os.path.join(path,label,file_name)
        if not os.path.exists(destination):
            source = os.path.join(input_dir + 'train',file_name)
            img=Image.open(source)
            new_image=img.resize((224,224))
            new_image.save(file_name)
            shutil.move(file_name,destination)

c:\users\dell\anaconda3\envs\deeplearning\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [25]:
train_path = '../Downloads/covid'
val_path = '../Downloads/19'
#------Generators------------#
train_augs = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=90,
    zoom_range=0.2, 
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.05,
    channel_shift_range=0.1
)


val_augs = ImageDataGenerator(rescale=1./255)

train_gen = train_augs.flow_from_directory(
    train_path,
    target_size=(224,224),
    batch_size=10,
    class_mode='binary')

val_gen = val_augs.flow_from_directory(
    val_path,
    target_size=(224,224),
    batch_size=10,
    class_mode='binary')
print(train_gen.class_indices)

Found 4211 images belonging to 2 classes.
Found 1053 images belonging to 2 classes.
{'Normal': 0, 'Pnemonia': 1}


In [26]:
base_model = VGG19(include_top=False,
                  input_shape = (224,224,3),
                  weights = 'imagenet')

for layer in base_model.layers[:-15]:
    layer.trainable = False
    
for layer in base_model.layers:
    print(layer,layer.trainable)

model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(1024,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))
model.summary()


<keras.engine.input_layer.InputLayer object at 0x000001FE948AF4A8> False
<keras.layers.convolutional.Conv2D object at 0x000001FE948AF390> False
<keras.layers.convolutional.Conv2D object at 0x000001FE948AF9B0> False
<keras.layers.pooling.MaxPooling2D object at 0x000001FE949FCEF0> False
<keras.layers.convolutional.Conv2D object at 0x000001FE948AFDD8> False
<keras.layers.convolutional.Conv2D object at 0x000001FE94A19630> False
<keras.layers.pooling.MaxPooling2D object at 0x000001FE94A55F98> False
<keras.layers.convolutional.Conv2D object at 0x000001FE94A554E0> True
<keras.layers.convolutional.Conv2D object at 0x000001FE94A87F98> True
<keras.layers.convolutional.Conv2D object at 0x000001FE94A9FBE0> True
<keras.layers.convolutional.Conv2D object at 0x000001FE94AD7A58> True
<keras.layers.pooling.MaxPooling2D object at 0x000001FE94AEAD30> True
<keras.layers.convolutional.Conv2D object at 0x000001FE94AEA7B8> True
<keras.layers.convolutional.Conv2D object at 0x000001FE94B406A0> True
<keras.lay

In [9]:
#-------Callbacks-------------#
checkpoint = ModelCheckpoint(
    './base.model',
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    mode='min',
    save_weights_only=False,
    period=1
)
earlystop = EarlyStopping(
    monitor='val_loss',
    min_delta=0.001,
    patience=10,
    verbose=1,
    mode='auto'
)
tensorboard = TensorBoard(
    log_dir = './logs',
    histogram_freq=0,
    batch_size=16,
    write_graph=True,
    write_grads=True,
    write_images=False,
)

csvlogger = CSVLogger(
    filename= "training_csv.log",
    separator = ",",
    append = False
)

#lrsched = LearningRateScheduler(step_decay,verbose=1)

reduce = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.3,
    patience=2,
    verbose=1, 
    mode='auto',
    cooldown=1 
)

callbacks = [checkpoint,tensorboard,csvlogger,reduce]

In [ ]:
#------------Train The Model----------#
#-------Use SGD with momentum and play with the learning rate and momentum----------#
#------Good Momentum values: 0.9,0.99,0.5----------#
opt = SGD(lr=1e-4,momentum=0.99)
opt1 = Adam(lr=2e-4)

model.compile(
    loss='binary_crossentropy',
    optimizer=opt,
    metrics=['accuracy']
)
    
history = model.fit_generator(
    train_gen, 
    steps_per_epoch  = 500, 
    validation_data  = val_gen,
    validation_steps = 500,
    epochs = 30, 
    verbose = 1,
    callbacks=callbacks
)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/30
500/500 [==============================] - 6802s 14s/step - loss: 0.4260 - accuracy: 0.7954 - val_loss: 0.1197 - val_accuracy: 0.8876

Epoch 00001: val_loss improved from inf to 0.11967, saving model to ./base.model

Epoch 2/30
500/500 [==============================] - 36527s 73s/step - loss: 0.2970 - accuracy: 0.8814 - val_loss: 0.8678 - val_accuracy: 0.7589

Epoch 00002: val_loss did not improve from 0.11967
Epoch 3/30
500/500 [==============================] - 6201s 12s/step - loss: 0.2310 - accuracy: 0.9109 - val_loss: 0.2581 - val_accuracy: 0.8892

Epoch 00003: val_loss did not improve from 0.11967

Epoch 00003: ReduceLROnPlateau reducing learning rate to 2.9999999242136255e-05.
Epoch 4/30
500/500 [==============================] - 6003s 12s/step - loss: 0.1987 - accuracy: 0.9242 - val_loss: 0.2444 - val_accuracy: 0.9302

Epoch 00004: val_loss did not improve from 0.11967
E